In [1]:
!python -m pip install django

In [3]:
!python -m django startproject config .

In [9]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is C6E9-1E28

 Directory of C:\Users\HP\OOP UCU\OOP\Django APP

05/12/2025  17:31    <DIR>          .
05/12/2025  17:25    <DIR>          ..
05/12/2025  17:25    <DIR>          .ipynb_checkpoints
05/12/2025  17:29    <DIR>          config
05/12/2025  17:30    <DIR>          finance_app
05/12/2025  17:29    <DIR>          loanmanager
05/12/2025  17:27               684 manage.py
05/12/2025  17:31             2,960 Untitled.ipynb
               2 File(s)          3,644 bytes
               6 Dir(s)  417,811,034,112 bytes free


In [ ]:
%cd "C:\Users\HP\OOP UCU\OOP\Django APP"

In [ ]:
import os
print(os.getcwd())

In [ ]:
!dir

In [ ]:
INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    "finance_app",  # 👈 make sure this is here
]

In [ ]:
from django.http import HttpResponse

def home(request):
    return HttpResponse("Django is working from Jupyter")

In [ ]:
!python manage.py startapp loanapp_999_xyz

In [ ]:
!python manage.py runserver

In [ ]:
manage.py

In [ ]:
%cd "C:\Users\HP\OOP UCU\OOP\Django APP"

In [ ]:
import os
print(os.getcwd())

In [ ]:
INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    "finance_app",   # <-- add this line
]

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.home, name="home"),
]

In [ ]:
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include("finance_app.urls")),  # root URL goes to finance_app
]

In [ ]:
!python manage.py migrate

In [ ]:
!python manage.py runserver

In [ ]:
from django.db import models


class Loan(models.Model):
    borrower_name = models.CharField(max_length=100)
    amount = models.DecimalField(max_digits=12, decimal_places=2)
    rate = models.DecimalField(max_digits=5, decimal_places=2)  # % per year
    duration_months = models.PositiveIntegerField()

    monthly_payment = models.DecimalField(max_digits=12, decimal_places=2)
    total_payment = models.DecimalField(max_digits=12, decimal_places=2)
    total_interest = models.DecimalField(max_digits=12, decimal_places=2)

    created_at = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return f"{self.borrower_name} - {self.amount}"

In [ ]:
!python manage.py makemigrations
!python manage.py migrate

In [ ]:
from django.shortcuts import render, redirect
from django.db.models import Q
from decimal import Decimal, InvalidOperation
from .models import Loan


def loan_list_create(request):
    """
    - GET: show form + list of loans (optionally filtered by search query)
    - POST: create a new loan record after computing payments
    """
    error = ""
    q = request.GET.get("q", "").strip()

    if request.method == "POST":
        borrower_name = request.POST.get("borrower_name", "").strip()
        amount_text = request.POST.get("amount", "").strip()
        rate_text = request.POST.get("rate", "").strip()
        duration_text = request.POST.get("duration_months", "").strip()

        # Basic validation
        if not borrower_name or not amount_text or not rate_text or not duration_text:
            error = "All fields are required."
        else:
            try:
                amount = Decimal(amount_text)
                rate = Decimal(rate_text)
                duration_months = int(duration_text)
            except (InvalidOperation, ValueError):
                error = "Amount, rate and duration must be valid numbers."
            else:
                if amount <= 0 or rate <= 0 or duration_months <= 0:
                    error = "All numeric values must be greater than zero."
                else:
                    # Loan calculation
                    monthly_rate = rate / Decimal("1200")  # % per year → decimal per month
                    one_plus_r_pow_n = (Decimal("1") + monthly_rate) ** duration_months
                    monthly_payment = (
                        amount * monthly_rate * one_plus_r_pow_n
                        / (one_plus_r_pow_n - Decimal("1"))
                    )
                    total_payment = monthly_payment * duration_months
                    total_interest = total_payment - amount

                    Loan.objects.create(
                        borrower_name=borrower_name,
                        amount=amount,
                        rate=rate,
                        duration_months=duration_months,
                        monthly_payment=monthly_payment.quantize(Decimal("0.01")),
                        total_payment=total_payment.quantize(Decimal("0.01")),
                        total_interest=total_interest.quantize(Decimal("0.01")),
                    )
                    return redirect("loan_list")

    # GET (or error case)
    loans = Loan.objects.all().order_by("-created_at")
    if q:
        loans = loans.filter(
            Q(borrower_name__icontains=q)
        )

    context = {
        "loans": loans,
        "error": error,
        "q": q,
    }
    return render(request, "loan_list.html", context)


def clear_loans(request):
    """Delete all loan records."""
    if request.method == "POST":
        Loan.objects.all().delete()
        return redirect("loan_list")
    return render(request, "confirm_clear.html")

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.loan_list_create, name="loan_list"),
    path("clear/", views.clear_loans, name="clear_loans"),
]

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.loan_list_create, name="loan_list"),
    path("clear/", views.clear_loans, name="clear_loans"),
]

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>{% block title %}Loan Manager{% endblock %}</title>
    <style>
        body { font-family: Arial, sans-serif; background: #f3f4f6; }
        .container {
            max-width: 800px;
            margin: 40px auto;
            background: white;
            padding: 20px;
            border-radius: 8px;
        }
        .error { color: red; }
        table { width: 100%; border-collapse: collapse; margin-top: 15px; }
        th, td { border: 1px solid #ddd; padding: 6px; text-align: center; }
        th { background: #e5e7eb; }
        input[type="text"], input[type="number"] {
            padding: 6px; border-radius: 4px; border: 1px solid #ccc;
        }
        button {
            padding: 6px 12px; border: none; border-radius: 4px;
            background: #4f46e5; color: white; cursor: pointer;
        }
        button:hover { background: #4338ca; }
    </style>
</head>
<body>
<div class="container">
    <h1>Loan Manager</h1>
    {% block content %}{% endblock %}
</div>
</body>
</html>

In [ ]:
{% extends "base.html" %}
{% block title %}Confirm Clear{% endblock %}

{% block content %}
<h2>Clear All Loans</h2>
<p>Are you sure you want to delete <strong>all loan records</strong>?</p>

<form method="post">
    {% csrf_token %}
    <button type="submit">Yes, clear all</button>
    <a href="{% url 'loan_list' %}">Cancel</a>
</form>
{% endblock %}

In [ ]:
!python manage.py runserver

In [ ]:
http://127.0.0.1:8000/